In [1]:
import requests

Plan:

- Fetch 10,000 players
- Fetch their battle logs (25 battles each)
- Parse decks from the battle logs, put them in a set (so we end up with only unique decks)
- *NOTE*: This script does not take into account decks with more than 8 cards (such as those used for Mega Draft/Whatever), and does not take into account tower troops. It would be easy to fix the former issue, and a little trickier to implement the latter, but I'll leave that to another day! :p 
(ok, the former issue i fixed later for my project by iterating over the decks and removing those with 8+ cards in a separate file, but that's so hacky that i am actually quite perturbed for writing this)


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

bearer = 'Bearer ' + os.getenv("API_KEY")
headers = {"Accept":"application/json", "authorization":bearer}

In [3]:
# Get top players for March 2024

r_player_tags = requests.get("https://proxy.royaleapi.dev/v1/locations/global/seasons/96/rankings/players?limit=10", headers=headers)

In [4]:
players = r_player_tags.json()["items"]

print(players)

[{'tag': '#J0VU9CGP', 'name': 'CAL Dominik', 'expLevel': 14, 'trophies': 8492, 'rank': 1, 'clan': {'tag': '#Q2YQR9VQ', 'name': '☆50☆SKCALALAS', 'badgeId': 16000029}}, {'tag': '#L8RUUCL0Y', 'name': 'Shoko❤️SFIX', 'expLevel': 14, 'trophies': 8439, 'rank': 2, 'clan': {'tag': '#QQRVV22R', 'name': '☆50 STARS☆', 'badgeId': 16000097}}, {'tag': '#Y9R22RQ2', 'name': 'CMG Ian77', 'expLevel': 14, 'trophies': 8434, 'rank': 3, 'clan': {'tag': '#L89GRCRR', 'name': 'Clan #L89GRCRR', 'badgeId': 16000154}}, {'tag': '#RVCQ2CQGJ', 'name': 'Stardust｜繁星✨', 'expLevel': 14, 'trophies': 8420, 'rank': 4, 'clan': {'tag': '#Q2RP2Q2V', 'name': 'CASINO ROYALE', 'badgeId': 16000029}}, {'tag': '#G9YV9GR8R', 'name': 'Mohamed Light', 'expLevel': 14, 'trophies': 8417, 'rank': 5, 'clan': {'tag': '#QRGR0CQ9', 'name': 'The Golden Wing', 'badgeId': 16000020}}, {'tag': '#2UJ2QQJJP', 'name': 'Takuma', 'expLevel': 14, 'trophies': 8397, 'rank': 6, 'clan': {'tag': '#Q0P8PY9V', 'name': 'top 1', 'badgeId': 16000014}}, {'tag': '#J

In [5]:
player_tags: set[str] = set()

In [6]:
for player in players:
    player_tags.add(player["tag"])

In [16]:
decks: list[list[str]] = []

In [17]:
players_info = []

In [ ]:
for tag in player_tags:
    players_info.append(requests.get("https://proxy.royaleapi.dev/v1/players/" + tag, headers=headers))

In [84]:
player_tags_list = [x for x in player_tags]

In [112]:
log_decks = []

for i, tag in enumerate(player_tags_list):
    print(f'[{i+1}/{len(player_tags_list)}] Fetching data for {tag}...')
    url_tag = tag.replace('#', '%23')
    r_bl = requests.get(f'https://proxy.royaleapi.dev/v1/players/{url_tag}/battlelog', headers=headers)
    bl = r_bl.json()
    print('Done! Fetching decks...')
    
    for battle in bl:
        log_deck = []
        for deck in battle['team'][0]['cards']:
            log_deck.append(deck['name'])
        log_deck.sort()
        log_decks.append(log_deck)
        log_deck = []
        for deck in battle['opponent'][0]['cards']:
            log_deck.append(deck['name'])
        log_deck.sort()
        log_decks.append(log_deck)
    
    print('Done! \n\n')

print('ALL OK!')

[1/9999] Fetching data for #9C9Y0R0G...
Done! Fetching decks...
Done! 


[2/9999] Fetching data for #PVCCVUL9L...
Done! Fetching decks...
Done! 


[3/9999] Fetching data for #C99RP0VR...
Done! Fetching decks...
Done! 


[4/9999] Fetching data for #2UGJQP08Y...
Done! Fetching decks...
Done! 


[5/9999] Fetching data for #LJCVGVG22...
Done! Fetching decks...
Done! 


[6/9999] Fetching data for #2L0UJQV0G...
Done! Fetching decks...
Done! 


[7/9999] Fetching data for #2G0CJ9L...
Done! Fetching decks...
Done! 


[8/9999] Fetching data for #YP9YCJLGJ...
Done! Fetching decks...
Done! 


[9/9999] Fetching data for #2Y9UJJ2UJ...
Done! Fetching decks...
Done! 


[10/9999] Fetching data for #2RQ80VUC8...
Done! Fetching decks...
Done! 


[11/9999] Fetching data for #LVRV0U922...
Done! Fetching decks...
Done! 


[12/9999] Fetching data for #VGUV89RY...
Done! Fetching decks...
Done! 


[13/9999] Fetching data for #2P8LPPU8P...
Done! Fetching decks...
Done! 


[14/9999] Fetching data for #GJ8RL9VL..

In [113]:
log_decks_set = set(tuple(x) for x in log_decks)

In [119]:
import pickle

In [120]:
with open('decks.pickle', 'wb') as file:
    pickle.dump(log_decks, file)

In [3]:
# Get card icons
r_pics = requests.get('https://proxy.royaleapi.dev/v1/cards', headers=headers)
pics = r_pics.json()

In [6]:
print(len(pics['items']))

110


In [8]:
for item in pics['items']:
    card_name = item['name']
    pic_url = item["iconUrls"]['medium']
    img_data = requests.get(pic_url).content

    with open(card_name+'.png', 'wb') as handler:
        handler.write(img_data)

In [9]:
card_list: list[str] = []

for item in pics['items']:
    card_list.append(item['name'])

In [10]:
print(card_list)

['Knight', 'Archers', 'Goblins', 'Giant', 'P.E.K.K.A', 'Minions', 'Balloon', 'Witch', 'Barbarians', 'Golem', 'Skeletons', 'Valkyrie', 'Skeleton Army', 'Bomber', 'Musketeer', 'Baby Dragon', 'Prince', 'Wizard', 'Mini P.E.K.K.A', 'Spear Goblins', 'Giant Skeleton', 'Hog Rider', 'Minion Horde', 'Ice Wizard', 'Royal Giant', 'Guards', 'Princess', 'Dark Prince', 'Three Musketeers', 'Lava Hound', 'Ice Spirit', 'Fire Spirit', 'Miner', 'Sparky', 'Bowler', 'Lumberjack', 'Battle Ram', 'Inferno Dragon', 'Ice Golem', 'Mega Minion', 'Dart Goblin', 'Goblin Gang', 'Electro Wizard', 'Elite Barbarians', 'Hunter', 'Executioner', 'Bandit', 'Royal Recruits', 'Night Witch', 'Bats', 'Royal Ghost', 'Ram Rider', 'Zappies', 'Rascals', 'Cannon Cart', 'Mega Knight', 'Skeleton Barrel', 'Flying Machine', 'Wall Breakers', 'Royal Hogs', 'Goblin Giant', 'Fisherman', 'Magic Archer', 'Electro Dragon', 'Firecracker', 'Mighty Miner', 'Elixir Golem', 'Battle Healer', 'Skeleton King', 'Archer Queen', 'Golden Knight', 'Monk', 